1. 인스타그램에 자동으로 로그인 한 후 “강남맛집추천”으로 검색하여 나오는 게시물 중 50개를
추출하여 사진과 해시태그를 수집하여 사진은 사진 폴더에 별도로 저장하고(사진파일의 이름은
1.jpg 부터 50.jpg) 해시태그는 txt파일 형식으로 저장하는 웹 크롤러를 저장하세요. (단 게시물의
사진이 여러 장 일 경우 대표사진 1 장만 수집하여 저장하면 됩니다)

iframe이기때문에 불러오지 못 하는 경우가 발생하기도 함  
눈에는 한페이지에 있는 것 처럼 보여도 각 카테고리가 분리되어 있음

In [ ]:
from selenium import webdriver
import time  
from bs4 import BeautifulSoup
import sys 
import math
import random
import unicodedata   # 인스타그램의 해시태그 수집 중 자음/모음 분리현상 방지용 모듈
import os

insta_id=input("1. 인스타그램 아이디를 입력해주세요:")
insta_pw=input("2. 인스타그램 패스워드를 입력해주세요:")
search_keyword = input("3.검색할 해쉬태그를 입력하세요(예: 강남맛집): ")
count = int(input('4.크롤링 할 건수는 몇건입니까?: '))
f_dir=input('5.파일이 저장될 경로만 쓰세요(기본경로 : c:\\temp\\ ) : ')
if f_dir =='' :
    f_dir = "c:\\temp\\"
    
s_time = time.time( )
dir_query = '인스타그램'
now = time.localtime()
time_ = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)
os.makedirs(f_dir+time_+'-'+search_keyword+'-'+search_keyword)
f_name=f_dir+time_+'-'+search_keyword+'-'+search_keyword+'\\'+time_+'-'+search_keyword+'.txt'

chrome_path = "c:/temp/chromedriver_91/chromedriver.exe"
driver = webdriver.Chrome(chrome_path)

driver.get("https://www.instagram.com/")
time.sleep(random.randrange(1,5))

#ID와 비번 입력후 로그인하기
eid = driver.find_element_by_name('username')
for a in insta_id :
        eid.send_keys(a)
        time.sleep(0.3)
epwd = driver.find_element_by_name('password')
for b in insta_pw :
        epwd.send_keys(b)
        time.sleep(0.5)

driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[3]/button/div').click()
time.sleep(5)

In [6]:
#로그인 정보 저장 거부, 알림 끄기
driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div/div/div/button').click()
time.sleep(2)
driver.find_element_by_xpath('/html/body/div[3]/div/div/div/div[3]/button[2]').click()
time.sleep(1)
#검색할 키워드 입력하기
element = driver.find_element_by_xpath('''//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/input''')

for c in search_keyword :
    element.send_keys(c)
    time.sleep(0.2)

time.sleep(5)
driver.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/div[3]/div/div[2]/div/div[1]').click()

In [7]:
def scroll_down(driver):
  driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
  time.sleep(5)
print('요청하신 데이터를 수집중이니 잠시만 기다려 주세요~^^')

import pandas as pd
urls=[]     # 인스타그램 URL 주소 저장할 리스트
urls_drop_dup=[]
scroll_cnt = math.ceil(count / 10)
a = 1
while (a <= scroll_cnt):
    scroll_down(driver) 
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    all = soup.find('article','KC1QD').find_all('a')
    
    for i in all:    
        url = i['href']
        urls.append(url)          
        urls_drop_dup = pd.Series(urls).drop_duplicates()
        
        if len(urls_drop_dup) >= count :
            break
    a+=1
    print('요청하신 데이터를 수집중이니 잠시만 더 기다려 주세요~^^')
    
    if len(urls_drop_dup) >= count :
        break
        
full_url=[]
url_cnt = 0
for i in range(0,len(urls_drop_dup)) :
    url = 'https://www.instagram.com' +urls[i]
    full_url.append(url)
    url_cnt += 1
    
    if url_cnt > count:
        break
print(full_url)

요청하신 데이터를 수집중이니 잠시만 기다려 주세요~^^
요청하신 데이터를 수집중이니 잠시만 더 기다려 주세요~^^
['https://www.instagram.com/p/CROfIHul-Qe/', 'https://www.instagram.com/p/CRQh1F-FUs0/', 'https://www.instagram.com/p/CRQIA8NlJuX/', 'https://www.instagram.com/p/CRRKQQLFJ63/', 'https://www.instagram.com/p/CRJF9-Jl-XF/', 'https://www.instagram.com/p/CRSxAExllkF/', 'https://www.instagram.com/p/CROejkFlZes/', 'https://www.instagram.com/p/CRREqPvlqCp/', 'https://www.instagram.com/p/CRRWKQJDUVB/', 'https://www.instagram.com/p/CRUBTC1nquO/']


In [30]:
import urllib.request
import urllib
import sys
import re

hash_txt = []    # 해쉬 태그 저장 
bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)

f_name=f_dir+time_+'-'+dir_query+'-'+search_keyword+'\\'+time_+'-'+search_keyword+'.txt'

file_no = 0                                
img_src2=[]


cur_count = 1
for c in range(0,len(full_url)) :
    driver.get(full_url[c])
    time.sleep(2)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    f = open(f_name, 'a',encoding='UTF-8')

    # 해당 페이지의 해시태그 수집
    tags = soup.find('div','EtaWk')

    try :
        tags_1 = tags.find_all('a')
    except :
        pass
    else :
        for d in range(0, len(tags_1)) :
            tags = tags_1[d].get_text()
            tags_11 = tags.translate(bmp_map)
            tags_2 = unicodedata.normalize('NFC', tags_11)
            #유니코드 문자열 tags_11 대한 정규화 형식 NFC로 반환
            for i in tags_2 :
                if i[0:1]=='#' :
                    hash_txt.append(tags_2)
                    print(tags_2)
                    f.write("\n" + str(tags_2))
    #사진 저장
    img_src = soup.find('div','KL4Bh').find_all('img')
    mg_src1=imgs.find('img')['src']
    except :
        continue
    else :
        img_name = img_dir +'\\'+ str(file_no)+'.jpg'
        urllib.request.urlretrieve(img_src1,img_name)       
        time.sleep(1)
        
        print("현재 %s 번째 게시물까지 저장완료했습니다=======" %file_no)
        print("\n")

        if file_no == cnt :
                break
        file_no += 1
    cur_count+=1

#카와카츠오토코
#합정맛집
#홍대맛집
#연남동맛집
#상수맛집
#망원동맛집
#서울맛집
#강남맛집
#강남역맛집
#강남가볼만한곳
#강남역파스타
#강남핫플
#강남술집
#청담맛집
#양재맛집
#논현맛집
#맛스타그램
#맛집
#인스타광고
#강남맛집
#거인통닭
#부산거인통닭
#부산치킨
#깡통시장
#부산깡통시장
#깡통시장맛집
#부평깡통시장
#자갈치시장
#자갈치시장맛집
#부산맛집
#부산여행
#원조부안집
#부안집
#잠실새내맛집
#잠실맛집
#잠실데이트
#한강공원
#잠실역맛집
#잠실새내역맛집
#한강피크닉
#코밥심_여의도
#여의도의축복
#아루히
#아루히스시
#진주집
#피양옥여의도점
#폭포수식당
#서울근교여행
#경기도맛집
#양주맛집
#백숙맛집
#피그웨이브_양주
#고소미부엌
#키토베지롤
#논현맛집
#논현동맛집
#논현
#논현역맛집
#강남
#강남역맛집
#강남맛집
#강남데이트
#논현데이트
#고구마
#고구마치즈돈까스
#고치돈
#키토김밥
#키토제닉식단
#키토식단
#치돈
#서울맛집추천
#고구마무스
#돈까스맛집
#미쁘동
#미쁘동
#미쁘동
#미쁘동
#연남동맛집
#연남동밥집
#카이센동
#홍대맛집
#우니미쁘동
#연남동데이트
#연어소바
#홍대데이트
#딱새우회
#데일리
#일상
#소통
#강남맛집
#강남역맛집
#삼겹살
#존맛탱
#숨은맛집
#고깃집열
#진짜급으로
#여행가기ㅠ
#주인공을찾아라
#스포애니역삼역점
#스포애니
#역삼역
#강남역
#강남
#강남헬스
#역삼
#역삼헬스
#강남맛집
#역삼맛집
#운동하는여자
#운동하는남자
#코로나
#백신
#운동
#헬스
#헬창
#원조부안집교대역점
#서초역
#서초동맛집
#서초동고기집
#반포맛집
#교대역맛집
#강남맛집
#교대맛집원조부안집
#교대맛집
#서초맛집
#강남핫플레이스
#강남핫플
#서울핫플
#소통
#데일리룩
#줌마소통
#육아소통
#육아맘
#줌마룩
#직장인룩
#출근룩코디
#데이트코디
#여행룩
#캐주얼룩
#미용인스타그램
#강남맛집
#청담맛집
#압구정맛집
#대구맛집
#수성못맛집
#앞산카페
#부산맛집
#센텀맛집
#광안리맛집
#샤넬
#샤넬지갑
#샤넬지퍼돌이
#샤넬지퍼돌이지갑
#샤넬지퍼카드지갑


In [25]:
file_no = 0                                
img_count = 1
img_src2=[]
for c in range(0,len(full_url)):
    driver.get(full_url[c])
    time.sleep(3)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    img_src = soup.find('div','KL4Bh').find_all('img')
    img_src2.append(img_src)

[<img alt="Photo by 나잘먹 on July 14, 2021. May be an image of food." class="FFVAD" crossorigin="anonymous" decoding="auto" style="visibility: hidden; object-fit: cover;"/>]
[<img alt="Photo by ⓒ코밥심 on July 13, 2021. May be an image of food and indoor." class="FFVAD" crossorigin="anonymous" decoding="auto" sizes="587.7875366210938px" src="https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/e35/s1080x1080/217099704_2913223015660324_7871510956040806476_n.jpg?_nc_ht=scontent-ssn1-1.cdninstagram.com&amp;_nc_cat=104&amp;_nc_ohc=oNfFhZxoH88AX9KNPUN&amp;edm=AABBvjUBAAAA&amp;ccb=7-4&amp;oh=1a776d706e2178bfee9a77b80e4f21e0&amp;oe=60F5F2F0&amp;_nc_sid=83d603" srcset="https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/sh0.08/e35/s640x640/217099704_2913223015660324_7871510956040806476_n.jpg?_nc_ht=scontent-ssn1-1.cdninstagram.com&amp;_nc_cat=104&amp;_nc_ohc=oNfFhZxoH88AX9KNPUN&amp;edm=AABBvjUBAAAA&amp;ccb=7-4&amp;oh=252b64ba4d51e3ed1278163f74f50b3f&amp;oe=60F5BD07&amp;_nc_sid=83d603 640w

[<img alt="&lt;피자메이트, 장안동&gt;
나이아가라 치즈폭포다. 역대급 폭포다. 유네스코 세계 치즈 유산이다. 치즈가 천장까지 늘어난다. 

얇고 바삭한 도우로 쌓여져있다. 안에는 압도적인 양의 치즈와 토마토 소스, 양송이 등 다양한 피자 토핑이 들어가있다. 빵사이 토핑이 풍족하게 들어있다. 풍족함의 정석. 한 조각 들어올리면 터져나오는 토핑이 인상적이다. 

치즈로 배부를 수 있다니 영광이다. 빵이 얇아서 더부룩한 느낌은 없다. 같이 나오는 핫소스, 갈릭디핑 소스가 좋다. 다만, 맥주나 사이드 메뉴를 팔지 않는게 좀 아쉽다. 올해 꼭 포장해서 집에서 맥주랑 먹어야지. 

📍 동대문구 장안동 329-3
◾M(6조각) 페페로니 / 양송이 ₩19,000
📷 @gyusun_e @boma___boma

#피자메이트 #피자 #피자맛집 #동대문맛집 #장안동맛집" class="FFVAD" crossorigin="anonymous" decoding="auto" style="visibility: hidden; object-fit: cover;"/>]
[<img alt="Photo by 성준 on July 13, 2021. May be an image of one or more people, biceps, people standing and indoor." class="FFVAD" crossorigin="anonymous" decoding="auto" style="object-fit: cover;"/>]
[<img alt="Photo by 훈이 on July 14, 2021. May be an image of text that says '강남&amp; 강남&amp;장안동 Host Bar 010 9241 7146 카톡 kies100 훈이실장 X'." class="FFVAD" crossorigin="anonymous" decoding="auto" style="object-fit: cover;"/>]


In [21]:
import urllib.request
import urllib
img_src=[]
for i in img_src2 :
    try :
        img_src1=i['src']
        print(img_src1)
        img_src.append(img_src1)
        img_count += 1
    except :
        continue

In [28]:
print(img_src2[1].find('src'))

AttributeError: ResultSet object has no attribute 'find'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [24]:
full_url

['https://www.instagram.com/p/CROfIHul-Qe/',
 'https://www.instagram.com/p/CRQh1F-FUs0/',
 'https://www.instagram.com/p/CRQIA8NlJuX/',
 'https://www.instagram.com/p/CRRKQQLFJ63/',
 'https://www.instagram.com/p/CRJF9-Jl-XF/',
 'https://www.instagram.com/p/CRSxAExllkF/',
 'https://www.instagram.com/p/CROejkFlZes/',
 'https://www.instagram.com/p/CRREqPvlqCp/',
 'https://www.instagram.com/p/CRRWKQJDUVB/',
 'https://www.instagram.com/p/CRUBTC1nquO/']